O churn pode decorrer e, consequentemente ser compreendido através da análise de expectativas negativas e eventuais frustrações do cliente com o produto ou serviço prestado desde o início do seu relacionamento.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [ ]:
#Dataset
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.shape

In [ ]:
df.head()

In [ ]:
# Data Clean

df.isna().sum().sum()

In [ ]:

#Verificando o balanceamento
target_count = df.Churn.value_counts()
print(target_count)

# Plot 
sns.countplot(df.Churn, palette = "OrRd")
plt.box(False)
plt.xlabel('Churn Não (0) / Sim (1)', fontsize = 11)
plt.ylabel('Total Sessões', fontsize = 11)
plt.title('Contagem de Classes\n')
plt.show()

In [ ]:
# Verificando Valores Únicos
df.nunique()

In [ ]:
# Listas vazias para os resultados
multiple_cols_cat = []
binary_cols = []

# Loop pelas colunas
for c in df.columns[1:]:
    if df.nunique()[c] == 2:
        binary_cols.append(c)
    else:
        multiple_cols_cat.append(c)

In [ ]:
multiple_cols_cat

In [ ]:
binary_cols

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)


sns.countplot(binary_cols[0], data=df, ax=axes[0,0])
sns.countplot(binary_cols[1], data=df, ax=axes[0,1])
sns.countplot(binary_cols[2], data=df, ax=axes[0,2])
sns.countplot(binary_cols[3], data=df, ax=axes[1,0])
sns.countplot(binary_cols[4], data=df, ax=axes[1,1])
sns.countplot(binary_cols[5], data=df, ax=axes[1,2])


In [ ]:
#Alterando o campo Churn para 0 ou 1

churn_numeric = {'Yes':1, 'No':0}
df.Churn.replace(churn_numeric, inplace=True)

In [ ]:
df[['gender','Churn']].groupby(['gender']).mean()
#O sexo está muito balanceado, então não parece ser um item que influenciaria no modelo

In [ ]:
table = pd.pivot_table(df, values='Churn', index=['gender'],
                    columns=['SeniorCitizen'], aggfunc=np.mean)
table

In [ ]:
sns.countplot("InternetService", data=df)

In [ ]:
df[['InternetService', 'Churn']].groupby('InternetService').mean()

In [ ]:
#Distribuição da variável Internet Service com as outras variáveis

fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
sns.countplot("StreamingTV", data=df, ax=axes[0,0])
sns.countplot("StreamingMovies", data=df, ax=axes[0,1])
sns.countplot("OnlineSecurity", data=df, ax=axes[0,2])
sns.countplot("OnlineBackup", data=df, ax=axes[1,0])
sns.countplot("DeviceProtection", data=df, ax=axes[1,1])
sns.countplot("TechSupport", data=df, ax=axes[1,2])

In [ ]:
df.PhoneService.value_counts()

In [ ]:
#Caso um cliente não tenha serviço de telefone ele não terá como ter múltiplas linhas telefônicas
#Como a coluna Multriplas linhas tem dados mais específicos não incluirei a coluna PhoneService, pois posso entender o número de pessoas que têm serviço telefônico na coluna MultipleLines.

df.MultipleLines.value_counts()

In [ ]:
df[['MultipleLines', 'Churn']].groupby('MultipleLines').mean()

### Contrato e meio de pagamento

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot("Contract", data=df)

In [ ]:
df[['Contract', 'Churn']].groupby('Contract').mean()

# Aparentemente quem faz contratos menores estão mais propensos a cancelar.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot("PaymentMethod", data=df)

In [ ]:
df[['PaymentMethod', 'Churn']].groupby('PaymentMethod').mean()

#Clientes que pagam com pagamento eletrônico tem uma maior probabilidade de cancelar, mas esse também é o método mais comum utilizado para pagamento.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 7))

sns.distplot(df["tenure"], ax = axes[0])
sns.distplot(df["MonthlyCharges"], ax = axes[1])

In [ ]:
df[['tenure', 'MonthlyCharges', 'Churn']].groupby('Churn').mean()

In [ ]:
df[['Contract', 'tenure']].groupby('Contract').mean()

### Após verificar as variáveis optei por utilizar as seguintes variáveis:

- Customer ID
- Gender
- PhoneService
- Contract
- TotalCharges

In [ ]:
df.drop(['customerID', 'gender', 'PhoneService', 'Contract', 'TotalCharges'], axis=1, inplace=True)

## Data Preprocessing

In [ ]:
df.nunique()

In [ ]:
# Listas vazias para os resultados
cat_features = []

# Loop pelas colunas
for c in df.columns[:-1]:
    if df.nunique()[c] <= 5:
        cat_features.append(c)

X = pd.get_dummies(df, columns = cat_features, drop_first = True)

In [ ]:
#Escalando as variáveis contínuas

sc = MinMaxScaler()
a = sc.fit_transform(df[['tenure']])
b = sc.fit_transform(df[['MonthlyCharges']])

X['tenure'] = a
X['MonthlyCharges'] = b

In [ ]:
X.shape

## Resampling

In [ ]:
sns.countplot('Churn', data=df).set_title('Distribuição da classe antes do Resampling')

In [ ]:
#Separando as classes positivas das negativas

X_no = X[X.Churn == 0]
X_yes = X[X.Churn == 1]

In [ ]:
#Upsampling para a classe positiva

X_yes_upsambled = X_yes.sample(n=len(X_no), replace=True)

print(len(X_yes_upsambled))

In [ ]:
#Combinando classes positivos e negativos e checando a classe de distribuição
X_upsampled = X_no.append(X_yes_upsambled).reset_index(drop = True)

sns.countplot('Churn', data = X_upsampled).set_title('Distribuição da classe após o Resampling')

## Criando e avaliando o modelo

In [ ]:
#Divisão de Treino e Teste

X = X_upsampled.drop(['Churn'], axis=1) #features (Variáveis indepedentes)
y = X_upsampled['Churn'] #target (Variáveis dependentes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

### Random Forest - GridSearchCV

In [ ]:
from time import time

# Usando um grid completo de todos os parâmetros
parameters = {"max_depth": [15, 20, 25, None],
              "max_features": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              'n_estimators': [150, 200, 250, 300]}

forest = RandomForestClassifier()

clf = GridSearchCV(estimator = forest, param_grid = parameters, n_jobs = -1, cv = 5)
start = time()
clf.fit(X, y)
print("GridSearchCV executou em %.2f segundos para todas as combinações de candidatos a parâmetros do modelo."
      % (time() - start))

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
pd.DataFrame(clf.cv_results_).head()

In [ ]:
previsoes = clf.predict(X_test)
previsoes

In [ ]:
print (accuracy_score(y_test, previsoes))